<a href="https://colab.research.google.com/github/BIM-Master/Hotel-Reviews/blob/EXPERIMENTAL!!!!/Hotel_reviews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
pip install nltk

In [ ]:
import nltk

In [ ]:
from nltk.tokenize import word_tokenize

In [ ]:
import pandas as pd

In [ ]:
Hotel_df = pd.read_csv("/content/Hotel_Reviews 2.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/Hotel_Reviews 2.csv'

In [ ]:
Hotel_df

Removing attributes

In [ ]:
columns_to_remove = ["Hotel_Address", "Additional_Number_of_Scoring", "Days_since_review", "lat", "lng"]

Hotel_df = Hotel_df.drop(columns=columns_to_remove, errors='ignore')

"Naming, Hotel_df cleaned & test it"

Tokenization

In [ ]:
nltk.download('punkt')

In [ ]:
Hotel_df['Negative_Review_tokenized'] = Hotel_df['Negative_Review'].apply(lambda x: word_tokenize(x))
Hotel_df['Positive_Review_tokenized'] = Hotel_df['Positive_Review'].apply(lambda x: word_tokenize(x))


In [ ]:
Hotel_df

Stop word removal

In [ ]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
def remove_stop_words(tokens):
    stop_words = set(stopwords.words('english'))
    stop_words.discard('no')
    stop_words.discard('not')
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens

Test if "No" and "Not" are not removed during stop words removal

In [ ]:
Hotel_df['Negative_Review_tokenized_SWR'] = Hotel_df['Negative_Review_tokenized'].apply(remove_stop_words)
Hotel_df['Positive_Review_tokenized_SWR'] = Hotel_df['Positive_Review_tokenized'].apply(remove_stop_words)

In [ ]:
Hotel_df

POS-tagging, might be redundant based on how the code looks after lemmitization

In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
from nltk import pos_tag

In [ ]:
Hotel_df['Negative_Review_tokenized_SWR_POS'] = Hotel_df['Negative_Review_tokenized_SWR'].apply(lambda x: pos_tag(x))
Hotel_df['Positive_Review_tokenized_SWR_POS'] = Hotel_df['Positive_Review_tokenized_SWR'].apply(lambda x: pos_tag(x))

In [ ]:
Hotel_df

Lemmatization part

In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
def lemmatize_tokens(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]

In [ ]:
def lemmatize_tokens(tagged_tokens):
    lemmatized_tokens = []
    for token, pos_tag in tagged_tokens:
        pos = pos_tag[0].lower() if pos_tag else 'n'
        pos = pos if pos in ['a', 'r', 'n', 'v'] else 'n'
        lemmatized_tokens.append(lemmatizer.lemmatize(token, pos))
    return lemmatized_tokens

In [ ]:
Hotel_df['Negative_Review_tokenized_SWR_POS_Lem'] = Hotel_df['Negative_Review_tokenized_SWR_POS'].apply(lemmatize_tokens)
Hotel_df['Positive_Review_tokenized_SWR_POS_Lem'] = Hotel_df['Positive_Review_tokenized_SWR_POS'].apply(lemmatize_tokens)

In [ ]:
Hotel_df

,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,...,Tags,days_since_review,Negative_Review_tokenized,Positive_Review_tokenized,Negative_Review_tokenized_SWR,Positive_Review_tokenized_SWR,Negative_Review_tokenized_SWR_POS,Positive_Review_tokenized_SWR_POS,Negative_Review_tokenized_SWR_POS_Lem,Positive_Review_tokenized_SWR_POS_Lem
0,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,"[I, am, so, angry, that, i, made, this, post, ...","[Only, the, park, outside, of, the, hotel, was...","[angry, made, post, available, via, possible, ...","[park, outside, hotel, beautiful]","[(angry, JJ), (made, VBD), (post, NN), (availa...","[(park, NN), (outside, JJ), (hotel, NN), (beau...","[angry, make, post, available, via, possible, ...","[park, outside, hotel, beautiful]"
1,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,...,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,"[No, Negative]","[No, real, complaints, the, hotel, was, great,...","[No, Negative]","[No, real, complaints, hotel, great, great, lo...","[(No, DT), (Negative, NNP)]","[(No, DT), (real, JJ), (complaints, NNS), (hot...","[No, Negative]","[No, real, complaint, hotel, great, great, loc..."
2,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,...,"[' Leisure trip ', ' Family with young childre...",3 days,"[Rooms, are, nice, but, for, elderly, a, bit, ...","[Location, was, good, and, staff, were, ok, It...","[Rooms, nice, elderly, bit, difficult, rooms, ...","[Location, good, staff, ok, cute, hotel, break...","[(Rooms, NNP), (nice, RB), (elderly, RB), (bit...","[(Location, NN), (good, JJ), (staff, NN), (ok,...","[Rooms, nice, elderly, bit, difficult, room, t...","[Location, good, staff, ok, cute, hotel, break..."
3,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,...,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,"[My, room, was, dirty, and, I, was, afraid, to...","[Great, location, in, nice, surroundings, the,...","[room, dirty, afraid, walk, barefoot, floor, l...","[Great, location, nice, surroundings, bar, res...","[(room, NN), (dirty, NN), (afraid, JJ), (walk,...","[(Great, NNP), (location, NN), (nice, JJ), (su...","[room, dirty, afraid, walk, barefoot, floor, l...","[Great, location, nice, surroundings, bar, res..."
4,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,...,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,"[You, When, I, booked, with, your, company, on...","[Amazing, location, and, building, Romantic, s...","[booked, company, line, showed, pictures, room...","[Amazing, location, building, Romantic, setting]","[(booked, VBN), (company, NN), (line, NN), (sh...","[(Amazing, VBG), (location, NN), (building, NN...","[book, company, line, show, picture, room, thi...","[Amazing, location, building, Romantic, setting]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,...,"[' Leisure trip ', ' Family with older childre...",704 day,"[no, trolly, or, staff, to, help, you, take, t...",[location],"[no, trolly, staff, help, take, luggage, room]",[location],"[(no, DT), (trolly, RB), (staff, NN), (help, N...","[(location, NN)]","[no, trolly, staff, help, take, luggage, room]",[location]
515734,8/22/2015,8.1,A

Feature Extraction, TD-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

# Initialize TfidfVectorizer with a limit on max features
vectorizer = TfidfVectorizer(max_features=5000)

# Fit and transform the negative reviews
Negative_Review_tokenized_SWR_POS_Lem_tfidf = vectorizer.fit_transform(Hotel_df['Negative_Review_tokenized_SWR_POS_Lem'].apply(' '.join))

# Fit and transform the positive reviews
Positive_Review_tokenized_SWR_POS_Lem_tfidf = vectorizer.fit_transform(Hotel_df['Positive_Review_tokenized_SWR_POS_Lem'].apply(' '.join))

# Stack the sparse matrices horizontally
tfidf_features = hstack([Negative_Review_tokenized_SWR_POS_Lem_tfidf, Positive_Review_tokenized_SWR_POS_Lem_tfidf])

# Optionally, you can convert the sparse matrix to a DataFrame if you really need to (be cautious with memory)
# tfidf_df = pd.DataFrame.sparse.from_spmatrix(tfidf_features, columns=vectorizer.get_feature_names_out())

# If you must concatenate with the original DataFrame
Hot_df = pd.concat([Hotel_df.reset_index(drop=True), pd.DataFrame.sparse.from_spmatrix(tfidf_features)], axis=1)

In [27]:
Hot_df

,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,...,9990,9991,9992,9993,9994,9995,9996,9997,9998,9999
0,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
515734,8/22/2015,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,Breakfast was ok and we got earlier check in,11,12,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
515735,8/19/2015,8.1,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienn...,19,2823,No Positive,0,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
515736,8/17/2015,8.1,Atlantis Hotel Vienna,Mexico,No Negative,0,2823,The rooms are enormous and really comfortable...,25,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Sentiment Analysis

In [28]:
import numpy as np

In [37]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk

# Download VADER lexicon
nltk.download('vader_lexicon')

# Initialize VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

def vader_sentiment(tokens):
    text = ' '.join(tokens)  # Join list of tokens into a single string
    score = sia.polarity_scores(text)
    return score['compound']

# Load your dataset (assuming it's already loaded as Hot_df)
# Example: Hot_df = pd.read_csv('path_to_your_dataset.csv')

# Apply sentiment analysis
Hotel_df['vader_sentiment_score_positive'] = Hotel_df['Positive_Review_tokenized_SWR_POS_Lem'].apply(vader_sentiment)
Hotel_df['vader_sentiment_score_negative'] = Hotel_df['Negative_Review_tokenized_SWR_POS_Lem'].apply(vader_sentiment)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [38]:
Hotel_df

,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,...,Negative_Review_tokenized,Positive_Review_tokenized,Negative_Review_tokenized_SWR,Positive_Review_tokenized_SWR,Negative_Review_tokenized_SWR_POS,Positive_Review_tokenized_SWR_POS,Negative_Review_tokenized_SWR_POS_Lem,Positive_Review_tokenized_SWR_POS_Lem,vader_sentiment_score_positive,vader_sentiment_score_negative
0,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,...,"[I, am, so, angry, that, i, made, this, post, ...","[Only, the, park, outside, of, the, hotel, was...","[angry, made, post, available, via, possible, ...","[park, outside, hotel, beautiful]","[(angry, JJ), (made, VBD), (post, NN), (availa...","[(park, NN), (outside, JJ), (hotel, NN), (beau...","[angry, make, post, available, via, possible, ...","[park, outside, hotel, beautiful]",0.5994,-0.8819
1,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,...,"[No, Negative]","[No, real, complaints, the, hotel, was, great,...","[No, Negative]","[No, real, complaints, hotel, great, great, lo...","[(No, DT), (Negative, NNP)]","[(No, DT), (real, JJ), (complaints, NNS), (hot...","[No, Negative]","[No, real, complaint, hotel, great, great, loc...",0.9477,-0.7096
2,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,...,"[Rooms, are, nice, but, for, elderly, a, bit, ...","[Location, was, good, and, staff, were, ok, It...","[Rooms, nice, elderly, bit, difficult, rooms, ...","[Location, good, staff, ok, cute, hotel, break...","[(Rooms, NNP), (nice, RB), (elderly, RB), (bit...","[(Location, NN), (good, JJ), (staff, NN), (ok,...","[Rooms, nice, elderly, bit, difficult, room, t...","[Location, good, staff, ok, cute, hotel, break...",0.8720,-0.4019
3,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,...,"[My, room, was, dirty, and, I, was, afraid, to...","[Great, location, in, nice, surroundings, the,...","[room, dirty, afraid, walk, barefoot, floor, l...","[Great, location, nice, surroundings, bar, res...","[(room, NN), (dirty, NN), (afraid, JJ), (walk,...","[(Great, NNP), (location, NN), (nice, JJ), (su...","[room, dirty, afraid, walk, barefoot, floor, l...","[Great, location, nice, surroundings, bar, res...",0.9260,-0.9394
4,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,...,"[You, When, I, booked, with, your, company, on...","[Amazing, location, and, building, Romantic, s...","[booked, company, line, showed, pictures, room...","[Amazing, location, building, Romantic, setting]","[(booked, VBN), (company, NN), (line, NN), (sh...","[(Amazing, VBG), (location, NN), (building, NN...","[book, company, line, show, picture, room, thi...","[Amazing, location, building, Romantic, setting]",0.7579,-0.2495
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,...,"[no, trolly, or, staff, to, help, you, take, t...",[location],"[no, trolly, staff, help, take, luggage, room]",[location],"[(no, DT), (trolly, RB), (staff, NN), (help, N...","[(location, NN)]","[no, trolly, staff, help, take, luggage, room]",[location],0.0000,0.1280
515734,8/22/2015,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,Breakfast was ok and we got earlier check in,11,12,...,"[The, hotel, looks, like, 3, but, surely, not, 4]","[Breakfast, was, ok, and, we, got, earlier,

In [40]:
!pip install afinn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.6/52.6 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53430 sha256=feca8e1d24f13f3a183ca0773d5d63b555b046f4b6af4111c869a34d69bf87e9
  Stored in directory: /root/.cache/pip/wheels/b0/05/90/43f79196199a138fb486902fceca30a2d1b5228e6d2db8eb90
Successfully built afinn


In [41]:
import pandas as pd
from afinn import Afinn

# Initialize Afinn
afinn = Afinn()

# Function to calculate AFINN sentiment score
def afinn_sentiment(tokens):
    text = ' '.join(tokens)  # Join list of tokens into a single string
    score = afinn.score(text)
    return score

Hotel_df['afinn_sentiment_score_positive'] = Hotel_df['Positive_Review_tokenized_SWR_POS_Lem'].apply(afinn_sentiment)
Hotel_df['afinn_sentiment_score_negative'] = Hotel_df['Negative_Review_tokenized_SWR_POS_Lem'].apply(afinn_sentiment)


In [42]:
Hotel_df

,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,...,Negative_Review_tokenized_SWR,Positive_Review_tokenized_SWR,Negative_Review_tokenized_SWR_POS,Positive_Review_tokenized_SWR_POS,Negative_Review_tokenized_SWR_POS_Lem,Positive_Review_tokenized_SWR_POS_Lem,vader_sentiment_score_positive,vader_sentiment_score_negative,afinn_sentiment_score_positive,afinn_sentiment_score_negative
0,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,...,"[angry, made, post, available, via, possible, ...","[park, outside, hotel, beautiful]","[(angry, JJ), (made, VBD), (post, NN), (availa...","[(park, NN), (outside, JJ), (hotel, NN), (beau...","[angry, make, post, available, via, possible, ...","[park, outside, hotel, beautiful]",0.5994,-0.8819,3.0,-4.0
1,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,...,"[No, Negative]","[No, real, complaints, hotel, great, great, lo...","[(No, DT), (Negative, NNP)]","[(No, DT), (real, JJ), (complaints, NNS), (hot...","[No, Negative]","[No, real, complaint, hotel, great, great, loc...",0.9477,-0.7096,9.0,-3.0
2,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,...,"[Rooms, nice, elderly, bit, difficult, rooms, ...","[Location, good, staff, ok, cute, hotel, break...","[(Rooms, NNP), (nice, RB), (elderly, RB), (bit...","[(Location, NN), (good, JJ), (staff, NN), (ok,...","[Rooms, nice, elderly, bit, difficult, room, t...","[Location, good, staff, ok, cute, hotel, break...",0.8720,-0.4019,8.0,0.0
3,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,...,"[room, dirty, afraid, walk, barefoot, floor, l...","[Great, location, nice, surroundings, bar, res...","[(room, NN), (dirty, NN), (afraid, JJ), (walk,...","[(Great, NNP), (location, NN), (nice, JJ), (su...","[room, dirty, afraid, walk, barefoot, floor, l...","[Great, location, nice, surroundings, bar, res...",0.9260,-0.9394,12.0,-11.0
4,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,...,"[booked, company, line, showed, pictures, room...","[Amazing, location, building, Romantic, setting]","[(booked, VBN), (company, NN), (line, NN), (sh...","[(Amazing, VBG), (location, NN), (building, NN...","[book, company, line, show, picture, room, thi...","[Amazing, location, building, Romantic, setting]",0.7579,-0.2495,4.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,...,"[no, trolly, staff, help, take, luggage, room]",[location],"[(no, DT), (trolly, RB), (staff, NN), (help, N...","[(location, NN)]","[no, trolly, staff, help, take, luggage, room]",[location],0.0000,0.1280,0.0,1.0
515734,8/22/2015,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,Breakfast was ok and we got earlier check in,11,12,...,"[hotel, looks, like, 3, surely, not, 4]","[Breakfast, ok, got, earlier, check]","[(hotel, NN), (looks, VBZ), (like, IN), (3, CD...","[(Breakfast, NNP), (ok, NN), (got, VBD), (earl...","[hotel, look, like, 3, surely, not, 4]","[Breakfast, ok, get, earlier, check]",0.2960,0.6597,0.0,2.0
515735,8/19/2015,8.1,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienn...,19,2823,No Positive,0,3,...,"[ac, useless, hot, week, vienna, gave, hot, air]","[No, Positive]","[(ac, JJ), (useless, JJ), (hot, JJ), (week, NN...","[(No, DT), (Positive, NNP)]","[ac, useless, ho

In [78]:
# Calculate the correlation matrix
correlation_matrix = Hotel_df[['Reviewer_Score', 'afinn_sentiment_score_positive', 'afinn_sentiment_score_negative']].corr()

# Print the correlation matrix
print(correlation_matrix)


                                Reviewer_Score  \
Reviewer_Score                        1.000000   
afinn_sentiment_score_positive        0.338151   
afinn_sentiment_score_negative        0.005686   

                                afinn_sentiment_score_positive  \
Reviewer_Score                                        0.338151   
afinn_sentiment_score_positive                        1.000000   
afinn_sentiment_score_negative                        0.049897   

                                afinn_sentiment_score_negative  
Reviewer_Score                                        0.005686  
afinn_sentiment_score_positive                        0.049897  
afinn_sentiment_score_negative                        1.000000  


In [77]:
correlation_matrix = Hotel_df[['Reviewer_Score', 'vader_sentiment_score_positive', 'vader_sentiment_score_negative']].corr()

# Print the correlation matrix
print(correlation_matrix)


                                Reviewer_Score  \
Reviewer_Score                        1.000000   
vader_sentiment_score_positive        0.376697   
vader_sentiment_score_negative       -0.054392   

                                vader_sentiment_score_positive  \
Reviewer_Score                                        0.376697   
vader_sentiment_score_positive                        1.000000   
vader_sentiment_score_negative                        0.000553   

                                vader_sentiment_score_negative  
Reviewer_Score                                       -0.054392  
vader_sentiment_score_positive                        0.000553  
vader_sentiment_score_negative                        1.000000  


In [79]:
correlation_matrix = Hotel_df[['afinn_sentiment_score_positive', 'vader_sentiment_score_positive']].corr()

# Print the correlation matrix
print(correlation_matrix)


                                afinn_sentiment_score_positive  \
afinn_sentiment_score_positive                        1.000000   
vader_sentiment_score_positive                        0.757252   

                                vader_sentiment_score_positive  
afinn_sentiment_score_positive                        0.757252  
vader_sentiment_score_positive                        1.000000  


In [47]:
# Create a new column that averages AFINN positive and negative scores
Hotel_df['afinn_avg_score'] = (Hotel_df['afinn_sentiment_score_positive'] + Hotel_df['afinn_sentiment_score_negative']) / 2


In [48]:
Hotel_df['vader_avg_score'] = (Hotel_df['vader_sentiment_score_positive'] + Hotel_df['vader_sentiment_score_negative']) / 2


In [81]:
correlation_matrix = Hotel_df[['Reviewer_Score','vader_avg_score', 'afinn_avg_score']].corr()

# Print the correlation matrix
print(correlation_matrix)

                 Reviewer_Score  vader_avg_score  afinn_avg_score
Reviewer_Score         1.000000         0.185515         0.297058
vader_avg_score        0.185515         1.000000         0.690047
afinn_avg_score        0.297058         0.690047         1.000000


In [51]:
power_distance_df = pd.read_csv('/content/power_distance_nationalities_final.csv', delimiter=';')
# string split the two relevant columns to ensure no unnecessary spaces at the start or end
Hotel_df['Reviewer_Nationality'] = Hotel_df['Reviewer_Nationality'].str.strip()
Hotel_df['Reviewer_Nationality'] = Hotel_df['Reviewer_Nationality'].str.strip()

# After cleaning, use merge function to add power distance as a new column with the
# corresponding pd value based on nationality and save it into a new data frame
Hotel_df_new = Hotel_df.merge(power_distance_df, on='Reviewer_Nationality', how='left')


In [53]:
Hotel_df_new

,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,...,Positive_Review_tokenized_SWR_POS,Negative_Review_tokenized_SWR_POS_Lem,Positive_Review_tokenized_SWR_POS_Lem,vader_sentiment_score_positive,vader_sentiment_score_negative,afinn_sentiment_score_positive,afinn_sentiment_score_negative,afinn_avg_score,vader_avg_score,Power_Distance
0,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,...,"[(park, NN), (outside, JJ), (hotel, NN), (beau...","[angry, make, post, available, via, possible, ...","[park, outside, hotel, beautiful]",0.5994,-0.8819,3.0,-4.0,-0.5,-0.14125,93.0
1,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,...,"[(No, DT), (real, JJ), (complaints, NNS), (hot...","[No, Negative]","[No, real, complaint, hotel, great, great, loc...",0.9477,-0.7096,9.0,-3.0,3.0,0.11905,28.0
2,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,...,"[(Location, NN), (good, JJ), (staff, NN), (ok,...","[Rooms, nice, elderly, bit, difficult, room, t...","[Location, good, staff, ok, cute, hotel, break...",0.8720,-0.4019,8.0,0.0,4.0,0.23505,38.0
3,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,...,"[(Great, NNP), (location, NN), (nice, JJ), (su...","[room, dirty, afraid, walk, barefoot, floor, l...","[Great, location, nice, surroundings, bar, res...",0.9260,-0.9394,12.0,-11.0,0.5,-0.00670,35.0
4,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,...,"[(Amazing, VBG), (location, NN), (building, NN...","[book, company, line, show, picture, room, thi...","[Amazing, location, building, Romantic, setting]",0.7579,-0.2495,4.0,-1.0,1.5,0.25420,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515733,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,...,"[(location, NN)]","[no, trolly, staff, help, take, luggage, room]",[location],0.0000,0.1280,0.0,1.0,0.5,0.06400,90.0
515734,8/22/2015,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,Breakfast was ok and we got earlier check in,11,12,...,"[(Breakfast, NNP), (ok, NN), (got, VBD), (earl...","[hotel, look, like, 3, surely, not, 4]","[Breakfast, ok, get, earlier, check]",0.2960,0.6597,0.0,2.0,1.0,0.47785,40.0
515735,8/19/2015,8.1,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienn...,19,2823,No Positive,0,3,...,"[(No, DT), (Positive, NNP)]","[ac, useless, hot, week, vienna, give, hot, air]","[No, Positive]",0.3400,-0.4215,1.0,-2.0,-0.5,-0.04075,70.0
515736,8/17/2015,8.1,Atlantis Hotel Vienna,Mexico,No Negative,0,2823,The rooms are enormous and really comfortable...,25,3,...,"[(rooms, NNS), (enormous, JJ), (really, RB), (...","[No, Negative]","[room, enormous, really, comfortable, believe,...",0.5563,-0.7096,2.0,-3.0,-0.5,-0.07665,81.0


In [82]:
correlation_matrix = Hotel_df_new[['Reviewer_Score','Power_Distance']].corr()

In [83]:
# Print the correlation matrix
print(correlation_matrix)

                Reviewer_Score  Power_Distance
Reviewer_Score         1.00000        -0.09376
Power_Distance        -0.09376         1.00000


In [58]:
import statsmodels.api as sm

From here on out we will check out the difference between

In [84]:
X = Hotel_df[['vader_sentiment_score_positive','vader_sentiment_score_negative']]  # IV: AFINN positive sentiment score
y = Hotel_df['Reviewer_Score']   # DV: Average Score
# Add a constant term to the independent variable
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()


In [125]:
X = Hotel_df[['afinn_sentiment_score_positive', 'afinn_sentiment_score_negative']]  # IVs: AFINN positive and negative sentiment scores
y = Hotel_df['Reviewer_Score']   # DV: Average Score

# Add interaction term
X['interaction_term'] = X['afinn_sentiment_score_positive'] * X['afinn_sentiment_score_negative']

# Add a constant term to the independent variables
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Print the summary of the regression
print(model.summary())

<ipython-input-125-c3cc342ea4c4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['interaction_term'] = X['afinn_sentiment_score_positive'] * X['afinn_sentiment_score_negative']


                            OLS Regression Results                            
Dep. Variable:         Reviewer_Score   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.118
Method:                 Least Squares   F-statistic:                 2.305e+04
Date:                Mon, 27 May 2024   Prob (F-statistic):               0.00
Time:                        12:46:11   Log-Likelihood:            -9.5382e+05
No. Observations:              515738   AIC:                         1.908e+06
Df Residuals:                  515734   BIC:                         1.908e+06
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

In [85]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         Reviewer_Score   R-squared:                       0.145
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                 4.369e+04
Date:                Mon, 27 May 2024   Prob (F-statistic):               0.00
Time:                        10:18:29   Log-Likelihood:            -9.4590e+05
No. Observations:              515738   AIC:                         1.892e+06
Df Residuals:                  515735   BIC:                         1.892e+06
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
const       

In [76]:
Hotel_df.describe()

,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,vader_sentiment_score_positive,vader_sentiment_score_negative,afinn_sentiment_score_positive,afinn_sentiment_score_negative,afinn_avg_score,vader_avg_score
count,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000,515738.000000
mean,8.397487,18.539450,2743.743944,17.776458,7.166001,8.395077,0.589229,-0.206575,5.267366,-0.783066,2.242150,0.191327
std,0.548048,29.690831,2317.464868,21.804185,11.040228,1.637856,0.340274,0.445124,5.119344,2.644570,2.939068,0.280218
min,5.200000,0.000000,43.000000,0.000000,1.000000,2.500000,-0.983200,-0.992600,-26.000000,-37.000000,-27.000000,-0.983200
25%,8.100000,2.000000,1161.000000,5.000000,1.000000,7.500000,0.361200,-0.709600,1.000000,-3.000000,0.000000,0.000000
50%,8.400000,9.000000,2134.000000,11.000000,3.000000,8.800000,0.690800,-0.153100,4.000000,0.000000,1.500000,0.159100
75%,8.800000,23.000000,3613.000000,22.000000,8.000000,9.600000,0.877900,0.000000,8.000000,0.000000,3.500000,0.395300
max,9.800000,408.000000,16670.000000,395.000000,355.000000,10.000000,0.998800,0.994600,85.000000,38.000000,44.000000,0.992300


In [132]:
Hotel_df_new.to_csv("Hotel_df_done", index=False)

In [131]:
Afinn_count_above_zero = (Hotel_df['afinn_avg_score'] > 0).sum()
Afinn_count_equal_zero = (Hotel_df['afinn_avg_score'] == 0).sum()
Afinn_count_below_zero = (Hotel_df['afinn_avg_score'] < 0).sum()

print("Number of rows with values equal 0 in column 'X':", Afinn_count_equal_zero)
print("Number of rows with values above 0 in column 'X':", Afinn_count_above_zero)
print("Number of rows with values bellow 0 in column 'X':", Afinn_count_below_zero)


Number of rows with values equal 0 in column 'X': 61695
Number of rows with values above 0 in column 'X': 380045
Number of rows with values bellow 0 in column 'X': 73998


In [122]:
Afinn_p_count_below_zero = (Hotel_df['afinn_sentiment_score_positive'] < 0).sum()
Afinn_p_count_equal_zero = (Hotel_df['afinn_sentiment_score_negative'] == 0).sum()

print("Number of rows with values equal or above 0 in column 'X':", Afinn_p_count_below_zero+Afinn_p_count_equal_zero)

Number of rows with values equal or above 0 in column 'X': 163929
